## Demultiplexing 8nt internal bridge adaptor library

Various methods tried to give best sensitivity and specifity 
* LAST aligner as done with 24nt library
* The tool, demultiplex used to assign barcodes where location of barcode maybe unknonw
* UMI-tools (still not done)

In [4]:
!echo $PATH
%cd /media/minion/minion_ssd1/iPSC_sci_poreC/no_sample/20201215_0817_MN22588_FAN30302_23732d3b/hac_output/

# The raw data was first basecalled and the outer barcodes, generated through the ONT 96 PCR multiplexing kit, called with `guppy`. 
# Raw FASTQ reads output to directories as follows 

!ls

/home/minion/miniconda3/bin:/home/minion/miniconda3/condabin:/sbin:/bin:/usr/bin:/usr/local/bin:/snap/bin
/media/minion/minion_ssd1/iPSC_sci_poreC/no_sample/20201215_0817_MN22588_FAN30302_23732d3b/hac_output
Sequence_curves_8nt_brigde_adapter.pdf	barcode51
Untitled.ipynb				barcode52
barcode01				barcode53
barcode02				barcode54
barcode03				barcode55
barcode04				barcode56
barcode05				barcode57
barcode06				barcode58
barcode07				barcode59
barcode08				barcode60
barcode09				barcode61
barcode10				barcode62
barcode11				barcode63
barcode12				barcode64
barcode13				barcode65
barcode14				barcode66
barcode15				barcode67
barcode16				barcode68
barcode17				barcode69
barcode18				barcode70
barcode19				barcode71
barcode20				barcode72
barcode21				barcode73
barcode22				barcode74
barcode23				barcode75
barcode24				barcode76
barcode25				barcode77
barcode26				barcode78
barcode27				barcode79
barcode28				barcode80
barcode29				barcode81
barcode30				barcode82
barcode31				barcode83
ba

## Mapping with LAST and error-matrix
Using the same error matrix as that used with 24nt.

Running following commands to map barcodes to all passed reads in the barcode37 directory. Barcode 37 refers to the outer barcode used when constructing the library with PCR multiplexing kit

Adapters include:

In [ ]:
# ouput of error matrix file for lastal
!cat /home/minion/test_in/last/ONT_24nt_BC.mat

# out of cat adapter fasta file for lastaldb database, barcodes 1 to 10 only but fasta holds all 96 8nt bridge adapters used
!cat /home/minion/test_in/last/8nt_bridge_adapter.fasta | head -n 20

The following commands used to aling barcodes to reads

```
lastdb -uNEAR '/home/minion/test_in/last/8nt_bridge_adapter.fasta' '/home/minion/test_in/last/8nt_bridge_adapter.fasta' lastal -Q 0 -P 10 -p /home/minion/test_in/last/ONT_24nt_BC.mat n home/minion/test_in/last/8nt_bridge_adapter.fasta barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37.fastq.gz > barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37.maf
```

Convert maf alignment file to tab-delineated and including only the read ID and assigned barcode. NB. This may well include multiple lines for each read as the same barcode or different barcodes are assigned

```
maf-convert -n tab barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37.maf | cut -f 2,7 | sort | uniq | gzip > barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37.barcode.assignment.txt.gz
```

In [5]:
# Number of reads from the input FASTQ, i.e. all Q>7 filtered reads within the barcode37 directory

!seqkit stat barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37.fastq.gz

# Output is 10,542,505 reads

file                                                                          format  type    num_seqs         sum_len  min_len  avg_len  max_len
barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37.fastq.gz  FASTQ   DNA   10,542,505  15,830,392,724      100  1,501.6   45,984


In [ ]:
# Can output the no. of barcodes assigned per barcode from the barcode_count file

!cat demultiplexed/barcodes_count.txt

In [4]:
!seqkit stat demultiplexed/reads_MKVII_F_R_*

file                                    format  type  num_seqs      sum_len  min_len  avg_len  max_len
demultiplexed/reads_MKVII_F_R_1.fq.gz   FASTQ   DNA     15,276   29,923,926      147  1,958.9    9,195
demultiplexed/reads_MKVII_F_R_10.fq.gz  FASTQ   DNA      8,725   16,952,318      222    1,943    8,580
demultiplexed/reads_MKVII_F_R_11.fq.gz  FASTQ   DNA     13,622   26,303,777      218    1,931    9,066
demultiplexed/reads_MKVII_F_R_12.fq.gz  FASTQ   DNA      5,465   10,739,245      178  1,965.1    7,635
demultiplexed/reads_MKVII_F_R_13.fq.gz  FASTQ   DNA     10,110   19,567,361      174  1,935.4    7,927
demultiplexed/reads_MKVII_F_R_14.fq.gz  FASTQ   DNA      9,521   18,547,288      246    1,948    9,624
demultiplexed/reads_MKVII_F_R_15.fq.gz  FASTQ   DNA     16,180   32,997,545      166  2,039.4    9,561
demultiplexed/reads_MKVII_F_R_16.fq.gz  FASTQ   DNA      6,154   12,400,998      246  2,015.1    8,755
demultiplexed/reads_MKVII_F_R_17.fq.gz  FASTQ   DNA      2,325    4,651,3

In [4]:
# Some reads may have multiple barcodes assigned to them
!echo "Number of assigned read and barcode pairs"
!gunzip -c /media/minion/minion_ssd1/iPSC_sci_poreC/no_sample/20201215_0817_MN22588_FAN30302_23732d3b/hac_output/barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37_barcode.assignment.txt.gz | wc -l

!echo

# The number of demultiplexed reads once removing reads that have multiple different barcodes assigned (conflicts)
!echo "Number of uniquely assigned reads"
!awk '{ sum+=$1;} END {print sum;}' /media/minion/minion_ssd1/iPSC_sci_poreC/no_sample/20201215_0817_MN22588_FAN30302_23732d3b/hac_output/demultiplexed/barcodes_count.txt

Number of assigned read and barcode pairs
1653958

Number of uniquely assigned reads
993428


In [5]:
# This as a % of input
!echo "Percentage of reads assigned unique barcode"
(993428 / 10542505) * 100

Percentage of reads assigned unique barcode


9.423073548459309

In [21]:
# Use Rscript /home/minion/length_plot.r to analyse the length distribution and number of reads assigned per barcode bin directory:

!ls demultiplexed/

barcodes_count.txt	     reads_MKVII_F_R_1.fq.gz
lengths_MKVII_F_R_1.txt.gz   reads_MKVII_F_R_10.fq.gz
lengths_MKVII_F_R_10.txt.gz  reads_MKVII_F_R_11.fq.gz
lengths_MKVII_F_R_11.txt.gz  reads_MKVII_F_R_12.fq.gz
lengths_MKVII_F_R_12.txt.gz  reads_MKVII_F_R_13.fq.gz
lengths_MKVII_F_R_13.txt.gz  reads_MKVII_F_R_14.fq.gz
lengths_MKVII_F_R_14.txt.gz  reads_MKVII_F_R_15.fq.gz
lengths_MKVII_F_R_15.txt.gz  reads_MKVII_F_R_16.fq.gz
lengths_MKVII_F_R_16.txt.gz  reads_MKVII_F_R_17.fq.gz
lengths_MKVII_F_R_17.txt.gz  reads_MKVII_F_R_18.fq.gz
lengths_MKVII_F_R_18.txt.gz  reads_MKVII_F_R_19.fq.gz
lengths_MKVII_F_R_19.txt.gz  reads_MKVII_F_R_2.fq.gz
lengths_MKVII_F_R_2.txt.gz   reads_MKVII_F_R_20.fq.gz
lengths_MKVII_F_R_20.txt.gz  reads_MKVII_F_R_21.fq.gz
lengths_MKVII_F_R_21.txt.gz  reads_MKVII_F_R_22.fq.gz
lengths_MKVII_F_R_22.txt.gz  reads_MKVII_F_R_23.fq.gz
lengths_MKVII_F_R_23.txt.gz  reads_MKVII_F_R_24.fq.gz
lengths_MKVII_F_R_24.txt.gz  reads_MKVII_F_R_25.fq.gz
lengths_MKVII_F_R_25.txt.gz  reads_

## Mapping with LAST and trained error matrix
As the error matrix was trained on dataset using 24nt ONT barcodes the next step was to try and train a error matrix using the 8nt internal bridge library

```
last-train -Q 0 -P 10 /home/minion/test_in/last/8nt_bridge_adapter.fasta barcode37/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_BC37.fastq.gz
```

This will give error. This pertain to the reference sequences being too short. Need to consult with Martin Frith about how to get this to work with the shorter barcodes. Have tried with the 24nt library and it works as expected. In this case for the 8nt library the 

```
lastal: batch size=8388608
lastal: calculating matrix probabilities...
lastal: matrix lambda=0.219722
lastal: matrix letter frequencies (upper=reference, lower=query):
A   C   G   T  
25  25  25  25 
25  25  25  25 
lastal: getting E-value parameters...
try temperature=1.07 OK
0.20384132514287567, 0.14069805173978259,
0.59669417995818008, -7.0809783103220845,
0.59669417995818008, -7.0809783103220845,
4.2092934902566794, -108.3344439674098,
4.2092934902566794, -108.3344439674098,
4.1413733516764175, -105.88931897852036
lastal: reading test_in/last/8nt_bridge_adapter.fasta...
lastal: reading /tmp/tmpiijan7vk...
lastal: bad FASTA sequence data: missing '>'
# lastal version: 982
# maximum percent identity: 100
# scale of score parameters: 4.5512
# scale used while training: 91.024

# lastal -j7 -S1 -P10 -v -r5 -q5 -a15 -b3 test_in/last/8nt_bridge_adapter.fasta /tmp/tmpiijan7vk

last-train: error: no alignments
```

## Mapping with LAST without error matrix
Tried to using same method but without using error matrix to guide substitution rates. Would expect this to decrease sensitivity but at this point would try anything

In [4]:
# Output with error matrix trained on library with 24nt ONT barcodes on a small subset of reads
!cat /home/minion/test_in/barcode_counts_8nt_barcode_adapter.txt

       68 MKVII_F_R_1
       29 MKVII_F_R_10
       46 MKVII_F_R_11
       25 MKVII_F_R_12
       49 MKVII_F_R_13
       49 MKVII_F_R_14
       58 MKVII_F_R_15
       30 MKVII_F_R_16
       15 MKVII_F_R_17
       80 MKVII_F_R_18
       30 MKVII_F_R_19
       50 MKVII_F_R_2
       60 MKVII_F_R_20
       96 MKVII_F_R_21
       44 MKVII_F_R_22
       39 MKVII_F_R_23
       64 MKVII_F_R_24
       21 MKVII_F_R_25
       67 MKVII_F_R_26
       25 MKVII_F_R_27
       13 MKVII_F_R_28
       63 MKVII_F_R_29
       53 MKVII_F_R_3
       17 MKVII_F_R_30
       49 MKVII_F_R_31
       28 MKVII_F_R_32
        2 MKVII_F_R_33
       29 MKVII_F_R_34
       18 MKVII_F_R_35
       71 MKVII_F_R_36
       35 MKVII_F_R_37
       65 MKVII_F_R_38
       38 MKVII_F_R_39
       42 MKVII_F_R_4
       43 MKVII_F_R_40
       33 MKVII_F_R_41
       36 MKVII_F_R_42
       14 MKVII_F_R_43
       19 MKVII_F_R_44
       52 MKVII_F_R_45
       13 MKVII_F_R_46
        6 MKVII_F_R_47
       35 MKVII_F_R_48
       18 MKVII

In [6]:
# And when not using error matrix on same subset of reads
!cat /home/minion/test_in/barcode_counts_8nt_barcode_adapter2.txt

# No appreciable difference seen

       67 MKVII_F_R_1
       28 MKVII_F_R_10
       44 MKVII_F_R_11
       21 MKVII_F_R_12
       49 MKVII_F_R_13
       45 MKVII_F_R_14
       58 MKVII_F_R_15
       30 MKVII_F_R_16
       15 MKVII_F_R_17
       76 MKVII_F_R_18
       36 MKVII_F_R_19
       48 MKVII_F_R_2
       59 MKVII_F_R_20
       85 MKVII_F_R_21
       38 MKVII_F_R_22
       37 MKVII_F_R_23
       63 MKVII_F_R_24
       21 MKVII_F_R_25
       66 MKVII_F_R_26
       25 MKVII_F_R_27
       16 MKVII_F_R_28
       63 MKVII_F_R_29
       47 MKVII_F_R_3
       31 MKVII_F_R_30
       48 MKVII_F_R_31
       24 MKVII_F_R_32
        1 MKVII_F_R_33
       45 MKVII_F_R_34
       48 MKVII_F_R_35
       66 MKVII_F_R_36
       34 MKVII_F_R_37
       52 MKVII_F_R_38
       38 MKVII_F_R_39
       42 MKVII_F_R_4
       40 MKVII_F_R_40
       54 MKVII_F_R_41
       33 MKVII_F_R_42
       12 MKVII_F_R_43
       20 MKVII_F_R_44
       52 MKVII_F_R_45
       32 MKVII_F_R_46
        6 MKVII_F_R_47
       74 MKVII_F_R_48
       18 MKVII

## Mapping with demultiplex tool

https://demultiplex.readthedocs.io/en/latest/ 

Has a sub command that allows for alignment when the location of barcodes is unknown. In the case of sc-MC this is exactly the situation that we have whereby internal barcodes are inserted at restriction sites that are then rejoined at unknown distance between eachother.

```
pip install demultiplex
```
The use for match subcommand as follows: 

```
demultiplex match barcodes.tsv reads.fastq 
```

In [4]:
!demultiplex match --help

usage: demultiplex match [-h] [-m MISMATCH] [-d] BARCODES INPUT

Demultiplex one file given a list of barcode tuples.

positional arguments:
  BARCODES     barcodes file
  INPUT        input file

optional arguments:
  -h, --help   show this help message and exit
  -m MISMATCH  number of mismatches (int default: 1)
  -d           use Levenshtein distance (default: False)


Try on a subset of data, 1 FASTQ to see efficiency with this tool: 

```
demultiplex match /home/minion/test_in/8nt_bridge_adapter.tsv /home/minion/test_in/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0.fastq
```

In [2]:
# The barcodes.tsv file includes Dpn2, barcode in sense orientation, EcoR1 site and then the barcode in the reverse complement. This equates to the full internal bridge adapter sequence

!less -S /home/minion/test_in/8nt_bridge_adapter.tsv | head

MKVII_F_R_1	GATCGAGCTCGAGAATTCTCGAGCTCGATC
MKVII_F_R_2	GATCACGGCCGGGAATTCCCGGCCGTGATC
MKVII_F_R_3	GATCACACGGAGGAATTCCTCCGTGTGATC
MKVII_F_R_4	GATCGGATACCGGAATTCCGGTATCCGATC
MKVII_F_R_5	GATCCATAGCGCGAATTCGCGCTATGGATC
MKVII_F_R_6	GATCACCACCACGAATTCGTGGTGGTGATC
MKVII_F_R_7	GATCAAGCCGGTGAATTCACCGGCTTGATC
MKVII_F_R_8	GATCCGATGCTCGAATTCGAGCATCGGATC
MKVII_F_R_9	GATCGTCGGCATGAATTCATGCCGACGATC
MKVII_F_R_10	GATCCCGTCAGAGAATTCTCTGACGGGATC


Using default settings which allow for 1 mismatch. Perhaps a little server results in few assigned reads 

Most reads assigned to unknonw FASTQ

In [7]:
!ls /home/minion/test_in/demultiplexed_8nt_bridge_adapter_3/

fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_1.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_10.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_11.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_13.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_14.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_15.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_18.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_2.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_21.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_22.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_24.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_26.fastq
fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_29.fastq
fastq_runid_67

In [8]:
!seqkit stat /home/minion/test_in/demultiplexed_8nt_bridge_adapter_3/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_*

file                                                                                                                                 format  type  num_seqs    sum_len  min_len  avg_len  max_len
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_3/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_1.fastq   FASTQ   DNA          1        709      709      709      709
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_3/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_10.fastq  FASTQ   DNA          1      1,795    1,795    1,795    1,795
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_3/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_11.fastq  FASTQ   DNA          4      6,173      892  1,543.3    2,330
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_3/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_13.fastq  FASTQ   DNA          1      1,652    1,652    1,652    1,652
/home/minion/test_in/demultipl

The stringency of the match can be reduced through an increased value for the `-m` flag 

```
demultiplex match -m 4 /home/minion/test_in/8nt_bridge_adapter.tsv /home/minion/test_in/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0.fastq
```

In [10]:
!seqkit stat /home/minion/test_in/demultiplexed_8nt_bridge_adapter_4/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_*

file                                                                                                                                 format  type  num_seqs    sum_len  min_len  avg_len  max_len
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_4/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_1.fastq   FASTQ   DNA          4      3,554      709    888.5    1,068
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_4/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_10.fastq  FASTQ   DNA          2      3,590    1,795    1,795    1,795
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_4/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_11.fastq  FASTQ   DNA         10     15,460      892    1,546    2,330
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_4/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_R_13.fastq  FASTQ   DNA          2      3,304    1,652    1,652    1,652
/home/minion/test_in/demultipl

In [15]:
# Try with just the 8nt indice itself without flanking sequence
!less -S /home/minion/test_in/rev2.tsv | head

# Just barcodes 1 to 10, this is the bottom strand sequence but this tool searchs both strands so is equivalent to supply forward or reverse sequences that make up the bridge

MKVII_R_1	CTCGAGCTC
MKVII_R_2	CCCGGCCGT
MKVII_R_3	CCTCCGTGT
MKVII_R_4	CCGGTATCC
MKVII_R_5	CGCGCTATG
MKVII_R_6	CGTGGTGGT
MKVII_R_7	CACCGGCTT
MKVII_R_8	CGAGCATCG
MKVII_R_9	CATGCCGAC
MKVII_R_10	CTCTGACGG


Run the command as follows

```
demultiplex match /home/minion/test_in/rev2.tsv /home/minion/test_in/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0.fastq
```

In [17]:
# This results in a lot of misasignment I guess because firstly it is too lenient and barcode sequence is too short

!seqkit stat /home/minion/test_in/demultiplexed_8nt_bridge_adapter_5/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_*

file                                                                                                                               format  type  num_seqs    sum_len  min_len  avg_len  max_len
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_5/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_R_1.fastq   FASTQ   DNA        472    783,032      312    1,659    5,110
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_5/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_R_10.fastq  FASTQ   DNA        324    540,681      286  1,668.8    5,110
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_5/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_R_11.fastq  FASTQ   DNA        151    222,350      317  1,472.5    3,491
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_5/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_R_12.fastq  FASTQ   DNA        167    254,995      303  1,526.9    4,410
/home/minion/test_in/demultiplexed_8nt_b

There are also a few codes were almost all reads are assigned to

Lastly we can try by supplying the fwd and rev separately as dual indices for ecah cell barcode. The match subcommand allows for the user to supply mutliple indices for each sample

So we may supply barcodes in the format:

*name* *index(fwd)* *index(rev)*

In [2]:
 !less -S /home/minion/test_in/8nt_fwd_rev.tsv | head

MKVII_F_1	GAGCTCGA	CTCGAGCTC		
MKVII_F_2	ACGGCCGG	CCCGGCCGT		
MKVII_F_3	ACACGGAG	CCTCCGTGT		
MKVII_F_4	GGATACCG	CCGGTATCC		
MKVII_F_5	CATAGCGC	CGCGCTATG		
MKVII_F_6	ACCACCAC	CGTGGTGGT		
MKVII_F_7	AAGCCGGT	CACCGGCTT		
MKVII_F_8	CGATGCTC	CGAGCATCG		
MKVII_F_9	GTCGGCAT	CATGCCGAC		
MKVII_F_10	CCGTCAGA	CTCTGACGG		


Run the following command:

```
demultiplex match /home/minion/test_in/8nt_fwd_rev.tsv /home/minion/test_in/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0.fastq
```

In [3]:
# Again this results in multiple assignments for a read and most reads being assigned to barcode 53

!seqkit stat /home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_*

file                                                                                                                               format  type  num_seqs    sum_len  min_len  avg_len  max_len
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_1.fastq   FASTQ   DNA         41     80,378      564  1,960.4    4,079
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_10.fastq  FASTQ   DNA         41     90,542      659  2,208.3    4,289
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_11.fastq  FASTQ   DNA         10     15,264      892  1,526.4    2,330
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_12.fastq  FASTQ   DNA         10     22,104      905  2,210.4    3,940
/home/minion/test_in/demultiplexed_8nt_b

In [ ]:
# If we take a random read ID from barcode 10 output FASTQ we can see the issue of multiple assignments 

!less -S /home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_10.fastq 

@8a1c3b2c-b5ad-4fa3-8ccb-cf71c0aa6920 runid=6729bc1ee463697649dc3809461a9ba1f551
ATTTGGCCAACTGGGTCAATTTACATTGACCACTGTTCATTTGCAGGTCATTCTTCAATATCATTAACCTTTCTGTTGGT
+
((*)'%%%&%%+$&&-'%$&)'$&%$%$###$#'%&($%$$($$$''.$$#'$%($%$#&'+(')2103356/899<87B
@b7747d14-3d72-4749-b5b8-49e3db36e2df runid=6729bc1ee463697649dc3809461a9ba1f551
AGTGGCCTTCAATTCCAGTTACGTATGTGCTCAGTGGTGCGAGATTCGCACCACTGCGATCTTCCTGTGGGGCTAACTGC
+
'+$$&)**),%)+2&+*,4713(((%%//2553<<;<96+*%+1185589:/(&##&&&(2122(&&0/254(+64&(''
@86df365b-48de-4eb8-ad3f-f00689d8b994 runid=6729bc1ee463697649dc3809461a9ba1f551
TTGTACTTGAATTTCAATTTACATGTTGCTCTCCCAAGTCCTTATTCTGAATGGCCACAGGAAAGCCAAACATTTCCTCA
+
(.--+(('$%%//,702031,---+6648:3=62.$&+4422647::532383221.*&00-3/'+++-*+58:/32:84
@97d12962-e8ac-4198-9a45-2ec8208c5e33 runid=6729bc1ee463697649dc3809461a9ba1f551
TTGTGCTGTTCAGTTTATCTTATTGCTGGTGCTGACACGATTGATGCTTGAAGTATCATTAACCTACTTGCCTGTCGCTC
+
#'%$$$'&$$##&%($$--.,'',)1*+5930)1&%$',(()&&2$''(''',9;;70021/1355::::;>98=77=;9
@b3209497-86e2-411e-

In [2]:
# grep for the top read ID @8a1c3b2c-b5ad-4fa3-8ccb-cf71c0aa6920

!grep "@8a1c3b2c-b5ad-4fa3-8ccb-cf71c0aa6920" -l /home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_*

/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_10.fastq
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_26.fastq
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_59.fastq
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_66.fastq
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_82.fastq
/home/minion/test_in/demultiplexed_8nt_bridge_adapter_6/fastq_runid_6729bc1ee463697649dc3809461a9ba1f551d4ac_0_0_MKVII_F_93.fastq


Can see that read `@8a1c3b2c-b5ad-4fa3-8ccb-cf71c0aa6920` has been assigned to binned FASTQ output for barcodes 10, 26, 59, 66, 82, and 93

I will give up with this tool as it is poorly documented and maintained. It does not seem to have any logic to incorporate information on similarity scores